<a href="https://colab.research.google.com/github/AsadiAhmad/Machine-Translation/blob/main/Code/%E2%85%AFa%E2%85%BDhine_Trans%E2%85%BCation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Install Libraries

In [ ]:
!pip install fasttext gensim numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 44.3 MB/s eta 0:00:00
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 10.3 MB/s eta 0:00:00
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313508 sha256=ea7bb42745cb32ced5d2cd9e416e8eade369f0f2a9e22dc3752e10a9ed66ee7b
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb77130

In [ ]:
# !pip install \
#     matplotlib==3.7.1 \
#     numpy==1.25 \
#     polars==1.20.0 \
#     scikit_learn==1.6.1 \
#     seaborn==0.13.2 \
#     cloud-tpu-client==0.10 \
#     torch==2.1.0 \
#     torchtext==0.16.0 \
#     torchvision==0.16.0 \
#     torchaudio==2.1.0 \
#     torchtune==0.5.0 \
#     torchdata==0.7.0

# Step 2: Import Libraries

In [71]:
from typing import List, Dict, Tuple

import gc
import random

import numpy as np
import polars as pol

from gensim.models import KeyedVectors
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils import clip_grad_norm_
from nltk.translate.bleu_score import corpus_bleu

# Step 3: Download Resources

## Download Dataset

In [ ]:
!wget https://raw.githubusercontent.com/AsadiAhmad/Machine-Translation/main/Dataset/OPUS_FR_EN.parquet -O OPUS_FR_EN.parquet

--2025-07-05 12:31:07--  https://raw.githubusercontent.com/AsadiAhmad/Machine-Translation/main/Dataset/OPUS_FR_EN.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 330148 (322K) [application/octet-stream]
Saving to: ‘OPUS_FR_EN.parquet’

OPUS_FR_EN.parquet  100%[===================>] 322.41K  --.-KB/s    in 0.04s   

2025-07-05 12:31:08 (7.00 MB/s) - ‘OPUS_FR_EN.parquet’ saved [330148/330148]



## Download FastText Model

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-aligned/wiki.en.align.vec
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-aligned/wiki.fr.align.vec

--2025-07-05 12:31:08--  https://dl.fbaipublicfiles.com/fasttext/vectors-aligned/wiki.en.align.vec
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.173.166.48, 18.173.166.74, 18.173.166.31, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.173.166.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5685446378 (5.3G) [binary/octet-stream]
Saving to: ‘wiki.en.align.vec’

wiki.en.align.vec   100%[===================>]   5.29G   108MB/s    in 45s     

2025-07-05 12:31:53 (122 MB/s) - ‘wiki.en.align.vec’ saved [5685446378/5685446378]

--2025-07-05 12:31:53--  https://dl.fbaipublicfiles.com/fasttext/vectors-aligned/wiki.fr.align.vec
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.173.166.74, 18.173.166.31, 18.173.166.51, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.173.166.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2605241176 (2.4G) [binary/octet-stream]
Savin

# Step 4: Loading Dataset

In [ ]:
raw_dataset = pol.read_parquet("OPUS_FR_EN.parquet")

# Step 5: Loading FastText

In [ ]:
embeddings_en = KeyedVectors.load_word2vec_format("wiki.en.align.vec")
embeddings_fr = KeyedVectors.load_word2vec_format("wiki.fr.align.vec")

In [ ]:
print("English vocab size:", len(embeddings_en.key_to_index))
print("French vocab size:", len(embeddings_fr.key_to_index))

English vocab size: 2519370
French vocab size: 1152449


In [ ]:
vector_hello = embeddings_en["hello"]

In [ ]:
print(vector_hello)

[-0.0376 -0.0431  0.0079  0.0444 -0.016  -0.0323 -0.0603  0.026   0.0408
  0.0123 -0.0055  0.0092 -0.0579 -0.051   0.0848 -0.0195  0.0287 -0.0632
  0.0238 -0.0322 -0.0219  0.1224  0.0419  0.0224 -0.0436 -0.0101  0.0033
  0.0397 -0.0438  0.0083 -0.0243  0.0424 -0.0125  0.017  -0.1008 -0.0274
 -0.0022  0.0335 -0.0538 -0.0041  0.0195 -0.1051 -0.0179 -0.034  -0.0195
  0.0475 -0.0225 -0.0261 -0.1105  0.0481 -0.0405 -0.1177  0.0684 -0.0243
  0.0481  0.1251 -0.06   -0.1225  0.0707  0.0423  0.0724  0.061  -0.0429
 -0.0241 -0.027  -0.0385 -0.0293  0.0775 -0.0545  0.0423  0.0234  0.0439
  0.0661  0.0433 -0.0411  0.0629 -0.0043  0.067   0.0409  0.0688  0.0194
  0.0438 -0.0367  0.055   0.0864 -0.0727 -0.0376  0.0953  0.0362 -0.0275
  0.033   0.0152  0.0521  0.0195  0.0302 -0.0158 -0.0054 -0.0454 -0.0129
  0.0886  0.1209 -0.0558 -0.0168  0.0191 -0.0756  0.0119 -0.0784  0.0052
  0.0236 -0.0643 -0.0172 -0.0864  0.0414 -0.0739 -0.0546 -0.0366  0.0758
  0.0292  0.0174  0.0486  0.1457  0.017  -0.0872  0

# Step 6: Config Polars Showing

In [ ]:
pol.Config.set_tbl_rows(-1)
pol.Config.set_tbl_cols(-1)
pol.Config.set_fmt_str_lengths(70)
pol.Config.set_fmt_table_cell_list_len(10)

polars.config.Config

In [ ]:
raw_dataset

text
str
"""(En 1991, par exemple, les tribunaux ont réexaminé 536 cas d'adoption …"
"""Ils se sont rencontrés sans moi. ###>They met without me."""
"""La Ministre Villedrouin, a insisté, dans son intervention, sur l’image…"
"""J'ai payé en liquide pour la semaine. ###>Pay cash week ."""
"""En revanche, un amendement du Viet Nam visant à supprimer le membre de…"
"""Le certificat peut être imprimé dans une ou plusieurs langues de la co…"
"""59. L’ASFC continue à se réserver le droit d’examiner les expéditions …"
"""Le soutien offert par l'APECA, par le biais du Programme de développem…"
"""L'autre question, d'une portée énorme, est le droit de veto. ###>The o…"


# Step 7: Detect available device

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(f"available device is: {device}")

available device is: cpu


# Step 8: Model Config

In [ ]:
class Config:
    DEVICE = device

    # # Model architecture
    # HIDDEN_DIM = 512
    # N_LAYERS = 1
    # DROPOUT = 0.5
    # TEACHER_FORCING_RATIO = 0.5

    # # Training
    # BATCH_SIZE = 32 if USE_TPU else 64  # Smaller batch for TPU memory
    # LEARNING_RATE = 0.001
    # N_EPOCHS = 10
    # CLIP = 1.0

    # # Evaluation
    # TOP_K = 5
    # BLEU_WEIGHTS = [0.25, 0.25, 0.25, 0.25]

    # # Data processing
    # MAX_SEQ_LEN = 50  # Limit sequence length
    # MIN_FREQ = 2      # Minimum word frequency

In [ ]:
config = Config()

In [ ]:
config.DEVICE

device(type='cpu')

# Step 9: Preprocessing

## Spliting columns

In [16]:
dataset_splited = (
    raw_dataset.with_columns(
        pol.col("text").str.split_exact("###>", 1).alias("split")
    )
    .unnest("split")
    .rename({"field_0": "french_text", "field_1": "english_text"})
    .with_columns(pol.int_range(0, pol.len()).alias("index"))
    .select(["index", "french_text", "english_text"])
)

In [17]:
dataset_splited.head(10)

index,french_text,english_text
i64,str,str
0,"""(En 1991, par exemple, les tribunaux ont réexaminé 536 cas d'adoption …","""(For instance, in 1991 the Polish courts reviewed 536 foreign adoption…"
1,"""Ils se sont rencontrés sans moi. ""","""They met without me."""
2,"""La Ministre Villedrouin, a insisté, dans son intervention, sur l’image…","""The minister Villedrouin, stressed in his speech on Haiti's image outs…"
3,"""J'ai payé en liquide pour la semaine. ""","""Pay cash week ."""
4,"""En revanche, un amendement du Viet Nam visant à supprimer le membre de…","""An amendment by Viet Nam, however, intended to delete the phrase “in c…"
5,"""Le certificat peut être imprimé dans une ou plusieurs langues de la co…","""The certifrcate can be printed in one or more of the languages of the …"
6,"""59. L’ASFC continue à se réserver le droit d’examiner les expéditions …","""59. The CBSA continues to reserve the right to examine shipments and c…"
7,"""Le soutien offert par l'APECA, par le biais du Programme de développem…","""ACOA's support, through the Community Futures Program to Community Bus…"
8,"""L'autre question, d'une portée énorme, est le droit de veto. ""","""The other issue, which is of enormous significance, is the right to th…"


In [18]:
print(f"total of rows of the dataset: {len(dataset_splited)}")

total of rows of the dataset: 2000


## Clean Garbage Characters and Digits

In [19]:
def clean_text(text: str) -> str:
    if not isinstance(text, str):
        return ""

    return (
        pol.col(text)
        .str.replace_all(r"[^\w\s]", " ")  # Remove special chars (keep letters/whitespace)
        .str.replace_all(r"\d", " ")       # Remove digits
        .str.replace_all(r"\s+", " ")      # Collapse multiple spaces
        .str.strip_chars()                 # Trim whitespace
    )

In [20]:
dataset_cleaned = dataset_splited.with_columns(
    clean_text("french_text").alias("french_text"),
    clean_text("english_text").alias("english_text")
)

In [21]:
dataset_cleaned.head(10)

index,french_text,english_text
i64,str,str
0,"""En par exemple les tribunaux ont réexaminé cas d adoption à l étranger…","""For instance in the Polish courts reviewed foreign adoption cases and …"
1,"""Ils se sont rencontrés sans moi""","""They met without me"""
2,"""La Ministre Villedrouin a insisté dans son intervention sur l image d …","""The minister Villedrouin stressed in his speech on Haiti s image outsi…"
3,"""J ai payé en liquide pour la semaine""","""Pay cash week"""
4,"""En revanche un amendement du Viet Nam visant à supprimer le membre de …","""An amendment by Viet Nam however intended to delete the phrase in case…"
5,"""Le certificat peut être imprimé dans une ou plusieurs langues de la co…","""The certifrcate can be printed in one or more of the languages of the …"
6,"""L ASFC continue à se réserver le droit d examiner les expéditions et l…","""The CBSA continues to reserve the right to examine shipments and conve…"
7,"""Le soutien offert par l APECA par le biais du Programme de développeme…","""ACOA s support through the Community Futures Program to Community Busi…"
8,"""L autre question d une portée énorme est le droit de veto""","""The other issue which is of enormous significance is the right to the …"


## Convert to Lowercase alphabet

In [22]:
def lower_text(text: str) -> str:
    return text.lower() if isinstance(text, str) else ""

In [23]:
dataset_lower = dataset_cleaned.with_columns(
    pol.col("french_text").map_elements(lower_text, return_dtype=pol.String).alias("french_text"),
    pol.col("english_text").map_elements(lower_text, return_dtype=pol.String).alias("english_text")
)

In [24]:
dataset_lower.head(10)

index,french_text,english_text
i64,str,str
0,"""en par exemple les tribunaux ont réexaminé cas d adoption à l étranger…","""for instance in the polish courts reviewed foreign adoption cases and …"
1,"""ils se sont rencontrés sans moi""","""they met without me"""
2,"""la ministre villedrouin a insisté dans son intervention sur l image d …","""the minister villedrouin stressed in his speech on haiti s image outsi…"
3,"""j ai payé en liquide pour la semaine""","""pay cash week"""
4,"""en revanche un amendement du viet nam visant à supprimer le membre de …","""an amendment by viet nam however intended to delete the phrase in case…"
5,"""le certificat peut être imprimé dans une ou plusieurs langues de la co…","""the certifrcate can be printed in one or more of the languages of the …"
6,"""l asfc continue à se réserver le droit d examiner les expéditions et l…","""the cbsa continues to reserve the right to examine shipments and conve…"
7,"""le soutien offert par l apeca par le biais du programme de développeme…","""acoa s support through the community futures program to community busi…"
8,"""l autre question d une portée énorme est le droit de veto""","""the other issue which is of enormous significance is the right to the …"


## tokenizing

In [25]:
def tokenize_text(text: str) -> List[str]:
    return text.split() if isinstance(text, str) else []

In [26]:
dataset_tokenized = dataset_lower.with_columns(
    pol.col("french_text").map_elements(tokenize_text, return_dtype=pol.List(pol.String)).alias("french_text"),
    pol.col("english_text").map_elements(tokenize_text, return_dtype=pol.List(pol.String)).alias("english_text")
)

In [27]:
dataset_tokenized.head(10)

index,french_text,english_text
i64,list[str],list[str]
0,"[""en"", ""par"", ""exemple"", ""les"", ""tribunaux"", ""ont"", ""réexaminé"", ""cas"", ""d"", … ""enfant""]","[""for"", ""instance"", ""in"", ""the"", ""polish"", ""courts"", ""reviewed"", ""foreign"", ""adoption"", … ""child""]"
1,"[""ils"", ""se"", ""sont"", ""rencontrés"", ""sans"", ""moi""]","[""they"", ""met"", ""without"", ""me""]"
2,"[""la"", ""ministre"", ""villedrouin"", ""a"", ""insisté"", ""dans"", ""son"", ""intervention"", ""sur"", … ""touristique""]","[""the"", ""minister"", ""villedrouin"", ""stressed"", ""in"", ""his"", ""speech"", ""on"", ""haiti"", … ""destination""]"
3,"[""j"", ""ai"", ""payé"", ""en"", ""liquide"", ""pour"", ""la"", ""semaine""]","[""pay"", ""cash"", ""week""]"
4,"[""en"", ""revanche"", ""un"", ""amendement"", ""du"", ""viet"", ""nam"", ""visant"", ""à"", … ""par""]","[""an"", ""amendment"", ""by"", ""viet"", ""nam"", ""however"", ""intended"", ""to"", ""delete"", … ""para""]"
5,"[""le"", ""certificat"", ""peut"", ""être"", ""imprimé"", ""dans"", ""une"", ""ou"", ""plusieurs"", … ""langues""]","[""the"", ""certifrcate"", ""can"", ""be"", ""printed"", ""in"", ""one"", ""or"", ""more"", … ""languages""]"
6,"[""l"", ""asfc"", ""continue"", ""à"", ""se"", ""réserver"", ""le"", ""droit"", ""d"", … ""comme""]","[""the"", ""cbsa"", ""continues"", ""to"", ""reserve"", ""the"", ""right"", ""to"", ""examine"", … ""as""]"
7,"[""le"", ""soutien"", ""offert"", ""par"", ""l"", ""apeca"", ""par"", ""le"", ""biais"", … ""collectivités""]","[""acoa"", ""s"", ""support"", ""through"", ""the"", ""community"", ""futures"", ""program"", ""to"", … ""development""]"
8,"[""l"", ""autre"", ""question"", ""d"", ""une"", ""portée"", ""énorme"", ""est"", ""le"", … ""veto""]","[""the"", ""other"", ""issue"", ""which"", ""is"", ""of"", ""enormous"", ""significance"", ""is"", … ""veto""]"


## Calculate Max Len

In [35]:
def calculate_max_len(dataset, column_name: str) -> int:
    if column_name not in ['french_text', 'english_text']:
        raise ValueError("Column name must be either 'french_text' or 'english_text'")

    max_len = 0
    for token_list in dataset[column_name]:
        current_len = len(token_list)
        if current_len > max_len:
            max_len = current_len

    return max_len

In [36]:
max_len_en = calculate_max_len(dataset_tokenized, "english_text")
max_len_fr = calculate_max_len(dataset_tokenized, "french_text")

In [37]:
print(f"French Max Len is: {max_len_fr} and English Max Len is: {max_len_en}")

French Max Len is: 231 and English Max Len is: 194


In [38]:
final_max_len = max(max_len_en, max_len_fr) + 2

## Add Special Tokens

In [39]:
def add_special_tokens(tokens, max_len: int, embeddings: KeyedVectors = None) -> List[str]:
    if tokens is None:
        return []
    if not isinstance(tokens, (list, pol.Series)):
        return []

    tokens = list(tokens.to_list() if isinstance(tokens, pol.Series) else tokens)

    if embeddings is not None:
        tokens = [word if word in embeddings else "<UNK>" for word in tokens]

    if len(tokens) > max_len - 2:
        tokens = tokens[:max_len - 2]
    tokens = ['<SOS>'] + tokens + ['<EOS>']
    if len(tokens) < max_len:
        tokens += ['<PAD>'] * (max_len - len(tokens))
    return tokens

In [40]:
dataset_taged = dataset_tokenized.with_columns(
    pol.col("french_text").map_elements(
        lambda x: add_special_tokens(x, max_len=final_max_len, embeddings=embeddings_fr),
        return_dtype=pol.List(pol.String)
    ).alias("french_text"),
    pol.col("english_text").map_elements(
        lambda x: add_special_tokens(x, max_len=final_max_len, embeddings=embeddings_en),
        return_dtype=pol.List(pol.String)
    ).alias("english_text")
)

In [41]:
dataset_taged.head(10)

index,french_text,english_text
i64,list[str],list[str]
0,"[""<SOS>"", ""en"", ""par"", ""exemple"", ""les"", ""tribunaux"", ""ont"", ""réexaminé"", ""cas"", … ""<PAD>""]","[""<SOS>"", ""for"", ""instance"", ""in"", ""the"", ""polish"", ""courts"", ""reviewed"", ""foreign"", … ""<PAD>""]"
1,"[""<SOS>"", ""ils"", ""se"", ""sont"", ""rencontrés"", ""sans"", ""moi"", ""<EOS>"", ""<PAD>"", … ""<PAD>""]","[""<SOS>"", ""they"", ""met"", ""without"", ""me"", ""<EOS>"", ""<PAD>"", ""<PAD>"", ""<PAD>"", … ""<PAD>""]"
2,"[""<SOS>"", ""la"", ""ministre"", ""<UNK>"", ""a"", ""insisté"", ""dans"", ""son"", ""intervention"", … ""<PAD>""]","[""<SOS>"", ""the"", ""minister"", ""villedrouin"", ""stressed"", ""in"", ""his"", ""speech"", ""on"", … ""<PAD>""]"
3,"[""<SOS>"", ""j"", ""ai"", ""payé"", ""en"", ""liquide"", ""pour"", ""la"", ""semaine"", … ""<PAD>""]","[""<SOS>"", ""pay"", ""cash"", ""week"", ""<EOS>"", ""<PAD>"", ""<PAD>"", ""<PAD>"", ""<PAD>"", … ""<PAD>""]"
4,"[""<SOS>"", ""en"", ""revanche"", ""un"", ""amendement"", ""du"", ""viet"", ""nam"", ""visant"", … ""<PAD>""]","[""<SOS>"", ""an"", ""amendment"", ""by"", ""viet"", ""nam"", ""however"", ""intended"", ""to"", … ""<PAD>""]"
5,"[""<SOS>"", ""le"", ""certificat"", ""peut"", ""être"", ""imprimé"", ""dans"", ""une"", ""ou"", … ""<PAD>""]","[""<SOS>"", ""the"", ""<UNK>"", ""can"", ""be"", ""printed"", ""in"", ""one"", ""or"", … ""<PAD>""]"
6,"[""<SOS>"", ""l"", ""asfc"", ""continue"", ""à"", ""se"", ""réserver"", ""le"", ""droit"", … ""<PAD>""]","[""<SOS>"", ""the"", ""cbsa"", ""continues"", ""to"", ""reserve"", ""the"", ""right"", ""to"", … ""<PAD>""]"
7,"[""<SOS>"", ""le"", ""soutien"", ""offert"", ""par"", ""l"", ""apeca"", ""par"", ""le"", … ""<PAD>""]","[""<SOS>"", ""acoa"", ""s"", ""support"", ""through"", ""the"", ""community"", ""futures"", ""program"", … ""<PAD>""]"
8,"[""<SOS>"", ""l"", ""autre"", ""question"", ""d"", ""une"", ""portée"", ""énorme"", ""est"", … ""<PAD>""]","[""<SOS>"", ""the"", ""other"", ""issue"", ""which"", ""is"", ""of"", ""enormous"", ""significance"", … ""<PAD>""]"


## Build Vocabulary

In [45]:
def build_vocab(
    dataset: pol.DataFrame,
    column_name: str,
    embeddings: KeyedVectors,
    verbose: bool = True
) -> tuple[dict, dict, dict]:
    """
    Build vocabulary from a Polars DataFrame column containing tokenized text with special tokens

    Args:
        dataset: Polars DataFrame containing tokenized text
        column_name: Name of column to process ('french_text' or 'english_text')
        embeddings: Pre-trained word vectors
        verbose: Whether to print statistics

    Returns:
        dict: Word to index mapping
        dict: Index to word mapping
        dict: Word frequency counts
    """
    # Initialize with special tokens
    special_tokens = ["<PAD>", "<SOS>", "<EOS>", "<UNK>"]
    vocab = {token: i for i, token in enumerate(special_tokens)}
    vocab_counts = {token: 0 for token in special_tokens}

    # Statistics tracking
    total_tokens = 0
    unique_words = set()
    words_in_embeddings = 0

    # First pass: collect all words and counts
    for token_list in dataset[column_name]:
        for word in token_list:
            total_tokens += 1
            unique_words.add(word)
            if word in vocab_counts:
                vocab_counts[word] += 1

    # Second pass: build vocabulary (excluding special tokens)
    idx = len(special_tokens)
    for word in unique_words:
        if word not in special_tokens:
            if word in embeddings:
                vocab[word] = idx
                words_in_embeddings += 1
                idx += 1
            else:
                # Track UNK words (they'll use the <UNK> token)
                if "<UNK>" in vocab_counts:
                    vocab_counts["<UNK>"] += 1

    # Create reverse vocabulary
    reverse_vocab = {i: word for word, i in vocab.items()}

    if verbose:
        vocab_size = len(vocab)
        embedding_coverage = words_in_embeddings / (len(unique_words) - len(special_tokens)) * 100

        print(f"\nVocabulary Report for '{column_name}':")
        print(f"- Total tokens processed: {total_tokens}")
        print(f"- Unique words: {len(unique_words)}")
        print(f"- Vocabulary size: {vocab_size}")
        print(f"- Words covered by embeddings: {words_in_embeddings}")
        print(f"- Embedding coverage: {embedding_coverage:.2f}%")
        print(f"- Special tokens counts:")
        for token in special_tokens:
            print(f"  {token}: {vocab_counts.get(token, 0)}")

    return vocab, reverse_vocab, vocab_counts

In [46]:
vocab_en, rev_vocab_en, counts_en = build_vocab(dataset_taged, "english_text", embeddings_en)
vocab_fr, rev_vocab_fr, counts_fr = build_vocab(dataset_taged, "french_text", embeddings_fr)


Vocabulary Report for 'english_text':
- Total tokens processed: 462000
- Unique words: 6983
- Vocabulary size: 6983
- Words covered by embeddings: 6979
- Embedding coverage: 100.00%
- Special tokens counts:
  <PAD>: 423266
  <SOS>: 2000
  <EOS>: 2000
  <UNK>: 168

Vocabulary Report for 'french_text':
- Total tokens processed: 462000
- Unique words: 8130
- Vocabulary size: 8130
- Words covered by embeddings: 8126
- Embedding coverage: 100.00%
- Special tokens counts:
  <PAD>: 418184
  <SOS>: 2000
  <EOS>: 2000
  <UNK>: 239


## Create Embedding Matrix

In [52]:
def create_embedding_matrix(
    vocab: Dict[str, int],
    embeddings: KeyedVectors,
    embedding_dim: int = 300,
    normalize: bool = True,
    verbose: bool = True
) -> np.ndarray:
    """
    Creates an embedding matrix from a vocabulary using pre-trained embeddings.
    Special tokens are handled intelligently with <UNK> initialized to average embedding.

    Args:
        vocab: Dictionary mapping words to indices
        embeddings: Pre-trained word vectors
        embedding_dim: Dimension of word embeddings
        normalize: Whether to L2-normalize vectors
        verbose: Whether to print detailed reports

    Returns:
        np.ndarray: Embedding matrix of shape (vocab_size, embedding_dim)
    """
    vocab_size = len(vocab)
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    special_tokens = ['<PAD>', '<SOS>', '<EOS>', '<UNK>']

    # Track statistics
    in_vocab_vectors = []
    words_not_found = []
    special_tokens_found = {token: False for token in special_tokens}

    # First pass: process all words and collect in-vocabulary vectors
    for word, idx in vocab.items():
        if word in special_tokens:
            special_tokens_found[word] = True
            continue

        if word in embeddings:
            vec = embeddings[word]
            if normalize:
                vec = vec / (np.linalg.norm(vec) + 1e-8)  # Safe normalization
            in_vocab_vectors.append(vec)
            embedding_matrix[idx] = vec
        else:
            words_not_found.append(word)

    # Compute average embedding for <UNK>
    avg_embedding = np.mean(in_vocab_vectors, axis=0) if in_vocab_vectors else np.zeros(embedding_dim)

    # Special token initialization strategies
    INIT_STRATEGY = {
        '<PAD>': lambda: np.zeros(embedding_dim),
        '<SOS>': lambda: np.random.normal(0, 0.1, embedding_dim),
        '<EOS>': lambda: np.random.normal(0, 0.1, embedding_dim),
        '<UNK>': lambda: avg_embedding
    }

    # Apply special token initialization
    for token, strategy in INIT_STRATEGY.items():
        if token in vocab:
            embedding_matrix[vocab[token]] = strategy()

    # Verbose reporting
    if verbose:
        print("\n=== Embedding Matrix Report ===")
        print(f"Vocabulary Size: {vocab_size}")
        print(f"Embedding Dimension: {embedding_dim}")
        print(f"Normalization: {'Enabled' if normalize else 'Disabled'}")
        print("\n[Coverage Statistics]")
        print(f"- Words found in FastText: {len(in_vocab_vectors)}")
        print(f"- Words not found: {len(words_not_found)}")
        print(f"- Special tokens: {sum(special_tokens_found.values())}/{len(special_tokens)}")

        if words_not_found:
            print("\n[OOV Words Sample]")
            print(', '.join(words_not_found[:10]) + ('...' if len(words_not_found) > 10 else ''))

        print("\n[Special Token Initialization]")
        for token in special_tokens:
            if token in vocab:
                vec = embedding_matrix[vocab[token]]
                print(f"- {token}: norm={np.linalg.norm(vec):.4f}, mean={np.mean(vec):.4f}")

        print("\n[Matrix Summary]")
        print(f"Shape: {embedding_matrix.shape}")
        print(f"Non-zero rows: {np.count_nonzero(np.linalg.norm(embedding_matrix, axis=1))}/{vocab_size}")
        print(f"Global mean norm: {np.mean(np.linalg.norm(embedding_matrix, axis=1)):.4f}")

    return embedding_matrix

In [53]:
embedding_matrix_en = create_embedding_matrix(vocab_en, embeddings_en)
embedding_matrix_fr = create_embedding_matrix(vocab_fr, embeddings_fr)


=== Embedding Matrix Report ===
Vocabulary Size: 6983
Embedding Dimension: 300
Normalization: Enabled

[Coverage Statistics]
- Words found in FastText: 6979
- Words not found: 0
- Special tokens: 4/4

[Special Token Initialization]
- <PAD>: norm=0.0000, mean=0.0000
- <SOS>: norm=1.7430, mean=-0.0053
- <EOS>: norm=1.7786, mean=0.0016
- <UNK>: norm=0.4094, mean=0.0008

[Matrix Summary]
Shape: (6983, 300)
Non-zero rows: 6982/6983
Global mean norm: 1.0000

=== Embedding Matrix Report ===
Vocabulary Size: 8130
Embedding Dimension: 300
Normalization: Enabled

[Coverage Statistics]
- Words found in FastText: 8126
- Words not found: 0
- Special tokens: 4/4

[Special Token Initialization]
- <PAD>: norm=0.0000, mean=0.0000
- <SOS>: norm=1.7974, mean=0.0010
- <EOS>: norm=1.6802, mean=-0.0070
- <UNK>: norm=0.4016, mean=-0.0013

[Matrix Summary]
Shape: (8130, 300)
Non-zero rows: 8129/8130
Global mean norm: 1.0000


## Convert tokens to numbers

In [54]:
def tokens_to_indices(
    token_lists: List[List[str]],
    vocab: Dict[str, int],
    unk_token: str = "<UNK>",
    verbose: bool = True
) -> List[List[int]]:
    """
    Convert tokenized sentences to numerical indices using a vocabulary.

    Args:
        token_lists: List of tokenized sentences (each sentence is a list of words)
        vocab: Dictionary mapping words to indices
        unk_token: Token to use for unknown words
        verbose: Whether to print conversion statistics

    Returns:
        List of lists containing numerical indices
    """
    if not isinstance(token_lists, list):
        raise ValueError("Input must be a list of tokenized sentences")

    indices = []
    unk_count = 0
    total_tokens = 0
    unk_words = set()

    for tokens in token_lists:
        sentence_indices = []
        for token in tokens:
            total_tokens += 1
            if token in vocab:
                sentence_indices.append(vocab[token])
            else:
                # Handle unknown words
                sentence_indices.append(vocab.get(unk_token, vocab["<UNK>"]))
                unk_count += 1
                unk_words.add(token)
        indices.append(sentence_indices)

    if verbose:
        print("\n=== Token Conversion Report ===")
        print(f"Processed {len(token_lists)} sentences")
        print(f"Total tokens: {total_tokens}")
        print(f"Unknown tokens: {unk_count} ({unk_count/max(1,total_tokens)*100:.2f}%)")
        print(f"Unique unknown words: {len(unk_words)}")
        if unk_words:
            print("Sample unknown words:", list(unk_words)[:10])

    return indices

In [55]:
french_indices = tokens_to_indices(
    dataset_taged["french_text"].to_list(),
    vocab_fr,
    unk_token="<UNK>"
)

english_indices = tokens_to_indices(
    dataset_taged["english_text"].to_list(),
    vocab_en,
    unk_token="<UNK>"
)


=== Token Conversion Report ===
Processed 2000 sentences
Total tokens: 462000
Unknown tokens: 0 (0.00%)
Unique unknown words: 0

=== Token Conversion Report ===
Processed 2000 sentences
Total tokens: 462000
Unknown tokens: 0 (0.00%)
Unique unknown words: 0


In [56]:
def add_indices_column(
    df: pol.DataFrame,
    text_column: str,
    vocab: Dict[str, int],
    new_column_name: str,
    unk_token: str = "<UNK>"
) -> pol.DataFrame:
    """
    Add a column with numerical indices to a Polars DataFrame
    """
    return df.with_columns(
        pol.col(text_column).map_elements(
            lambda x: tokens_to_indices([x], vocab, unk_token, verbose=False)[0],
            return_dtype=pol.List(pol.Int64)
        ).alias(new_column_name)
    )

In [57]:
dataset_indexed = add_indices_column(dataset_taged, "french_text", vocab_fr, "french_indices")
dataset_indexed = add_indices_column(dataset_indexed, "english_text", vocab_en, "english_indices")

In [58]:
dataset_indexed.head(10)

index,french_text,english_text,french_indices,english_indices
i64,list[str],list[str],list[i64],list[i64]
0,"[""<SOS>"", ""en"", ""par"", ""exemple"", ""les"", ""tribunaux"", ""ont"", ""réexaminé"", ""cas"", … ""<PAD>""]","[""<SOS>"", ""for"", ""instance"", ""in"", ""the"", ""polish"", ""courts"", ""reviewed"", ""foreign"", … ""<PAD>""]","[1, 4597, 5657, 616, 5622, 1733, 4061, 3760, 5151, … 0]","[1, 2050, 3765, 3608, 1842, 1196, 2620, 3198, 2178, … 0]"
1,"[""<SOS>"", ""ils"", ""se"", ""sont"", ""rencontrés"", ""sans"", ""moi"", ""<EOS>"", ""<PAD>"", … ""<PAD>""]","[""<SOS>"", ""they"", ""met"", ""without"", ""me"", ""<EOS>"", ""<PAD>"", ""<PAD>"", ""<PAD>"", … ""<PAD>""]","[1, 6136, 673, 1735, 4161, 2780, 1769, 2, 0, … 0]","[1, 2705, 1932, 4588, 5782, 2, 0, 0, 0, … 0]"
2,"[""<SOS>"", ""la"", ""ministre"", ""<UNK>"", ""a"", ""insisté"", ""dans"", ""son"", ""intervention"", … ""<PAD>""]","[""<SOS>"", ""the"", ""minister"", ""villedrouin"", ""stressed"", ""in"", ""his"", ""speech"", ""on"", … ""<PAD>""]","[1, 2451, 6944, 3, 6243, 3720, 4604, 2893, 5822, … 0]","[1, 1842, 3889, 2143, 82, 3608, 2791, 536, 52, … 0]"
3,"[""<SOS>"", ""j"", ""ai"", ""payé"", ""en"", ""liquide"", ""pour"", ""la"", ""semaine"", … ""<PAD>""]","[""<SOS>"", ""pay"", ""cash"", ""week"", ""<EOS>"", ""<PAD>"", ""<PAD>"", ""<PAD>"", ""<PAD>"", … ""<PAD>""]","[1, 7517, 658, 2544, 4597, 2498, 4480, 2451, 2330, … 0]","[1, 5796, 2984, 1739, 2, 0, 0, 0, 0, … 0]"
4,"[""<SOS>"", ""en"", ""revanche"", ""un"", ""amendement"", ""du"", ""viet"", ""nam"", ""visant"", … ""<PAD>""]","[""<SOS>"", ""an"", ""amendment"", ""by"", ""viet"", ""nam"", ""however"", ""intended"", ""to"", … ""<PAD>""]","[1, 4597, 6410, 2308, 7529, 5887, 2245, 2252, 6688, … 0]","[1, 3898, 416, 4314, 2004, 2012, 3219, 5758, 18, … 0]"
5,"[""<SOS>"", ""le"", ""certificat"", ""peut"", ""être"", ""imprimé"", ""dans"", ""une"", ""ou"", … ""<PAD>""]","[""<SOS>"", ""the"", ""<UNK>"", ""can"", ""be"", ""printed"", ""in"", ""one"", ""or"", … ""<PAD>""]","[1, 4837, 4521, 7343, 6542, 4319, 4604, 7760, 6377, … 0]","[1, 1842, 3, 2152, 333, 676, 3608, 3880, 3851, … 0]"
6,"[""<SOS>"", ""l"", ""asfc"", ""continue"", ""à"", ""se"", ""réserver"", ""le"", ""droit"", … ""<PAD>""]","[""<SOS>"", ""the"", ""cbsa"", ""continues"", ""to"", ""reserve"", ""the"", ""right"", ""to"", … ""<PAD>""]","[1, 1296, 4463, 5111, 5599, 673, 1680, 4837, 5187, … 0]","[1, 1842, 2683, 3981, 18, 5544, 1842, 2742, 18, … 0]"
7,"[""<SOS>"", ""le"", ""soutien"", ""offert"", ""par"", ""l"", ""apeca"", ""par"", ""le"", … ""<PAD>""]","[""<SOS>"", ""acoa"", ""s"", ""support"", ""through"", ""the"", ""community"", ""futures"", ""program"", … ""<PAD>""]","[1, 4837, 1500, 2931, 5657, 1296, 6392, 5657, 4837, … 0]","[1, 1756, 6442, 6781, 5709, 1842, 3274, 6626, 4582, … 0]"
8,"[""<SOS>"", ""l"", ""autre"", ""question"", ""d"", ""une"", ""portée"", ""énorme"", ""est"", … ""<PAD>""]","[""<SOS>"", ""the"", ""other"", ""issue"", ""which"", ""is"", ""of"", ""enormous"", ""significance"", … ""<PAD>""]","[1, 1296, 6642, 7122, 1979, 7760, 4229, 3759, 871, … 0]","[1, 1842, 2803, 4413, 5644, 4863, 6002, 5977, 1725, … 0]"


## Spliting into Train and Validation

In [60]:
def train_validation_split(
    df: pol.DataFrame,
    validation_ratio: float = 0.1
) -> Tuple[pol.DataFrame, pol.DataFrame]:

    if not 0 <= validation_ratio <= 1:
        raise ValueError("validation_ratio must be between 0 and 1")

    split_point = int(len(df) * (1 - validation_ratio))

    train_set = df.head(split_point)
    validation_set = df.tail(len(df) - split_point)

    return train_set, validation_set

In [61]:
train_set, validation_set = train_validation_split(dataset_indexed, 0.1)

In [62]:
print(f"len of train set is {len(train_set)} and the len of the validation set is {len(validation_set)}")

len of train set is 1800 and the len of the validation set is 200


## Create Dataset and DataLoader

In [65]:
class BilingualDataset(Dataset):
    def __init__(
        self,
        df: pol.DataFrame,
        src_vocab: Dict[str, int],
        tgt_vocab: Dict[str, int],
        src_lang: str = "french",
        tgt_lang: str = "english",
        pad_idx: int = 0
    ):
        """
        PyTorch Dataset for bilingual text pairs.

        Args:
            df: Polars DataFrame containing tokenized text
            src_vocab: Source language vocabulary
            tgt_vocab: Target language vocabulary
            src_lang: Name of source language column
            tgt_lang: Name of target language column
            pad_idx: Padding token index
        """
        self.df = df
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab
        self.src_lang = src_lang
        self.tgt_lang = tgt_lang
        self.pad_idx = pad_idx

        # Validate columns exist
        if src_lang not in df.columns or tgt_lang not in df.columns:
            raise ValueError(f"DataFrame must contain '{src_lang}' and '{tgt_lang}' columns")

    def __len__(self) -> int:
        return len(self.df)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        src_tokens = self.df[self.src_lang][idx]
        tgt_tokens = self.df[self.tgt_lang][idx]

        # Convert tokens to indices
        src_indices = [self.src_vocab.get(token, self.src_vocab["<UNK>"]) for token in src_tokens]
        tgt_indices = [self.tgt_vocab.get(token, self.tgt_vocab["<UNK>"]) for token in tgt_tokens]

        return (
            torch.tensor(src_indices, dtype=torch.long),
            torch.tensor(tgt_indices, dtype=torch.long)
        )

def create_dataloaders(
    train_df: pol.DataFrame,
    val_df: pol.DataFrame,
    src_vocab: Dict[str, int],
    tgt_vocab: Dict[str, int],
    batch_size: int = 32,
    src_lang: str = "french",
    tgt_lang: str = "english",
    pad_idx: int = 0,
    num_workers: int = 4
) -> Tuple[DataLoader, DataLoader]:
    """
    Create training and validation DataLoaders.

    Args:
        train_df: Training DataFrame
        val_df: Validation DataFrame
        src_vocab: Source language vocabulary
        tgt_vocab: Target language vocabulary
        batch_size: Batch size
        src_lang: Source language column name
        tgt_lang: Target language column name
        pad_idx: Padding token index
        num_workers: Number of workers for data loading

    Returns:
        Tuple of (train_loader, val_loader)
    """
    # Create datasets
    train_dataset = BilingualDataset(
        train_df, src_vocab, tgt_vocab, src_lang, tgt_lang, pad_idx
    )
    val_dataset = BilingualDataset(
        val_df, src_vocab, tgt_vocab, src_lang, tgt_lang, pad_idx
    )

    # Custom collate function for padding
    def collate_fn(batch):
        src_seqs, tgt_seqs = zip(*batch)

        # Calculate max lengths in this batch
        src_len = max(len(seq) for seq in src_seqs)
        tgt_len = max(len(seq) for seq in tgt_seqs)

        # Pad sequences
        src_padded = torch.full((len(batch), src_len), pad_idx, dtype=torch.long)
        tgt_padded = torch.full((len(batch), tgt_len), pad_idx, dtype=torch.long)

        for i, (src, tgt) in enumerate(batch):
            src_padded[i, :len(src)] = src
            tgt_padded[i, :len(tgt)] = tgt

        return src_padded, tgt_padded

    # Create DataLoaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn,
        num_workers=num_workers,
        pin_memory=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=collate_fn,
        num_workers=num_workers,
        pin_memory=True
    )

    # Print statistics
    print(f"\nCreated DataLoaders:")
    print(f"- Training batches: {len(train_loader)}")
    print(f"- Validation batches: {len(val_loader)}")
    print(f"- Batch size: {batch_size}")
    print(f"- Source language: {src_lang}")
    print(f"- Target language: {tgt_lang}")

    return train_loader, val_loader

In [66]:
# Create DataLoaders
train_loader, val_loader = create_dataloaders(
    train_df=train_set,
    val_df=validation_set,
    src_vocab=vocab_fr,
    tgt_vocab=vocab_en,
    batch_size=64,
    src_lang="french_text",
    tgt_lang="english_text",
    pad_idx=vocab_en["<PAD>"],  # Assuming same PAD index for both
    num_workers=4
)

# # Example usage in training loop
# for batch_idx, (src, tgt) in enumerate(train_loader):
#     # src/tgt shapes: (batch_size, seq_len)
#     # Your training code here
#     if batch_idx == 0:
#         print("\nFirst batch sample:")
#         print(f"Source shape: {src.shape}")
#         print(f"Target shape: {tgt.shape}")
#         print(f"Source sample:\n{src[0]}")
#         print(f"Target sample:\n{tgt[0]}")


Created DataLoaders:
- Training batches: 29
- Validation batches: 4
- Batch size: 64
- Source language: french_text
- Target language: english_text


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


# Step 10: Encoding Layer

In [77]:
class Encoder(nn.Module):
    def __init__(
        self,
        input_dim: int,
        emb_dim: int,
        enc_hid_dim: int,
        pretrained_embeddings: torch.Tensor = None,
        dropout: float = 0.5
    ):
        """
        Encoder with pretrained embeddings and conceptual space projection

        Args:
            input_dim: Source vocabulary size
            emb_dim: Embedding dimension (300 for FastText)
            enc_hid_dim: Hidden dimension for LSTM
            pretrained_embeddings: Preloaded embedding matrix
            dropout: Dropout probability
        """
        super().__init__()

        # Embedding layer
        self.embedding = nn.Embedding(input_dim, emb_dim)
        if pretrained_embeddings is not None:
            self.embedding.weight.data.copy_(pretrained_embeddings)
            self.embedding.weight.requires_grad = False  # Freeze embeddings

        # Conceptual space projection
        self.conceptual_proj = nn.Linear(emb_dim, emb_dim)
        self.conceptual_norm = nn.LayerNorm(emb_dim)

        # Bidirectional LSTM
        self.rnn = nn.LSTM(
            emb_dim,
            enc_hid_dim,
            num_layers=1,
            bidirectional=True,
            dropout=dropout if dropout > 0 else 0
        )

        # Hidden state projection to conceptual space
        self.fc_hidden = nn.Linear(2 * enc_hid_dim, emb_dim)
        self.fc_cell = nn.Linear(2 * enc_hid_dim, emb_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, src: torch.Tensor) -> Tuple[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:
        """
        Forward pass of encoder

        Args:
            src: Source sequence tensor (batch_size, src_len)

        Returns:
            Tuple of:
                - Conceptual representations (batch_size, src_len, emb_dim)
                - Tuple of hidden and cell states (batch_size, emb_dim)
        """
        # src shape: (batch_size, src_len)
        embedded = self.dropout(self.embedding(src))
        # embedded shape: (batch_size, src_len, emb_dim)

        # Project to conceptual space
        conceptual_embedded = self.conceptual_norm(
            F.gelu(self.conceptual_proj(embedded))
        )

        # LSTM processing
        outputs, (hidden, cell) = self.rnn(conceptual_embedded)
        # outputs shape: (batch_size, src_len, 2*enc_hid_dim)
        # hidden/cell shape: (2, batch_size, enc_hid_dim)

        # Combine bidirectional hidden states
        hidden = self.fc_hidden(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        cell = self.fc_cell(torch.cat((cell[-2,:,:], cell[-1,:,:]), dim=1))
        # hidden/cell shape: (batch_size, emb_dim)

        return conceptual_embedded, (hidden.unsqueeze(0), cell.unsqueeze(0))

# Step 11: Decoding Layer

In [73]:
class Decoder(nn.Module):
    def __init__(
        self,
        output_dim: int,
        emb_dim: int,
        dec_hid_dim: int,
        pretrained_embeddings: torch.Tensor = None,
        dropout: float = 0.5,
        teacher_forcing_ratio: float = 0.5
    ):
        """
        Decoder with teacher forcing and conceptual space projection

        Args:
            output_dim: Target vocabulary size
            emb_dim: Embedding dimension (must match encoder)
            dec_hid_dim: Hidden dimension for LSTM
            pretrained_embeddings: Preloaded embedding matrix
            dropout: Dropout probability
            teacher_forcing_ratio: Probability of using teacher forcing
        """
        super().__init__()

        self.output_dim = output_dim
        self.teacher_forcing_ratio = teacher_forcing_ratio

        # Embedding layer
        self.embedding = nn.Embedding(output_dim, emb_dim)
        if pretrained_embeddings is not None:
            self.embedding.weight.data.copy_(pretrained_embeddings)
            self.embedding.weight.requires_grad = False  # Freeze embeddings

        # Conceptual space projection
        self.conceptual_proj = nn.Linear(emb_dim, emb_dim)
        self.conceptual_norm = nn.LayerNorm(emb_dim)

        # LSTM layer
        self.rnn = nn.LSTM(
            emb_dim,
            dec_hid_dim,
            num_layers=1,
            dropout=dropout if dropout > 0 else 0
        )

        # Projection to vocabulary space
        self.fc_out = nn.Linear(dec_hid_dim, output_dim)

        # Alignment with encoder's conceptual space
        self.attention = nn.Linear(dec_hid_dim + emb_dim, 1)

        self.dropout = nn.Dropout(dropout)

    def forward(
        self,
        trg: torch.Tensor,
        encoder_concepts: torch.Tensor,
        hidden: torch.Tensor,
        cell: torch.Tensor
    ) -> Tuple[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:
        """
        Forward pass of decoder

        Args:
            trg: Target sequence tensor (batch_size, trg_len)
            encoder_concepts: Encoder's conceptual representations (batch_size, src_len, emb_dim)
            hidden: Previous hidden state (1, batch_size, emb_dim)
            cell: Previous cell state (1, batch_size, emb_dim)

        Returns:
            Tuple of:
                - Output predictions (batch_size, trg_len, output_dim)
                - Tuple of final hidden and cell states
        """
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]

        # Initialize outputs
        outputs = torch.zeros(trg_len, batch_size, self.output_dim).to(trg.device)

        # Process each timestep
        for t in range(1, trg_len):
            # Get input (use teacher forcing with probability)
            use_teacher_forcing = random.random() < self.teacher_forcing_ratio
            input = trg[:, t-1] if use_teacher_forcing else outputs[t-1].argmax(1)

            # Embed and project to conceptual space
            embedded = self.dropout(self.embedding(input.unsqueeze(0)))
            conceptual_embedded = self.conceptual_norm(
                F.gelu(self.conceptual_proj(embedded))
            )
            # LSTM step
            output, (hidden, cell) = self.rnn(conceptual_embedded, (hidden, cell))

            # Attention mechanism (simple additive attention)
            attn_weights = torch.softmax(
                self.attention(torch.cat((hidden.squeeze(0), encoder_concepts.mean(1)), dim=1)),
                dim=1
            )
            attended = (attn_weights.unsqueeze(2) * encoder_concepts).sum(dim=1)

            # Final prediction
            output = self.fc_out(torch.cat((output.squeeze(0), attended), dim=1))
            outputs[t] = output

        return outputs.permute(1, 0, 2), (hidden, cell)

# Step 12: Training Function

In [74]:
def train_model(
    encoder: nn.Module,
    decoder: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    src_vocab: dict,
    tgt_vocab: dict,
    optimizer: optim.Optimizer,
    criterion: nn.Module,
    device: torch.device,
    n_epochs: int = 10,
    max_grad_norm: float = 1.0,
    checkpoint_path: str = None,
    patience: int = 3
):
    """
    Training loop with progress tracking and memory optimization

    Args:
        encoder: Encoder model
        decoder: Decoder model
        train_loader: Training DataLoader
        val_loader: Validation DataLoader
        src_vocab: Source vocabulary
        tgt_vocab: Target vocabulary
        optimizer: Optimizer
        criterion: Loss function
        device: Target device (cuda/cpu)
        n_epochs: Number of epochs
        max_grad_norm: Gradient clipping norm
        checkpoint_path: Path to save checkpoints
        patience: Early stopping patience
    """
    # Free memory by deleting FastText embeddings
    global embeddings_en, embeddings_fr
    del embeddings_en, embeddings_fr
    gc.collect()

    # Create reverse vocabularies for decoding
    tgt_idx2word = {i: w for w, i in tgt_vocab.items()}
    pad_idx = tgt_vocab['<PAD>']
    sos_idx = tgt_vocab['<SOS>']
    eos_idx = tgt_vocab['<EOS>']

    # Track best validation loss for early stopping
    best_val_loss = float('inf')
    epochs_without_improvement = 0

    # Training history
    history = {
        'train_loss': [], 'train_acc': [], 'train_bleu': [],
        'val_loss': [], 'val_acc': [], 'val_bleu': []
    }

    for epoch in range(1, n_epochs + 1):
        # Training phase
        encoder.train()
        decoder.train()
        epoch_train_loss = 0
        epoch_train_correct = 0
        epoch_train_total = 0
        references = []
        hypotheses = []

        with tqdm(train_loader, unit="batch", desc=f"Epoch {epoch}/{n_epochs} [Train]") as t:
            for src, trg in t:
                src, trg = src.to(device), trg.to(device)

                # Forward pass
                optimizer.zero_grad()
                encoder_concepts, (hidden, cell) = encoder(src)
                outputs, _ = decoder(trg, encoder_concepts, hidden, cell)

                # Calculate loss (ignore padding)
                loss = criterion(
                    outputs.view(-1, outputs.shape[-1]),
                    trg.view(-1)
                )

                # Backward pass
                loss.backward()
                clip_grad_norm_(encoder.parameters(), max_grad_norm)
                clip_grad_norm_(decoder.parameters(), max_grad_norm)
                optimizer.step()

                # Calculate accuracy
                preds = outputs.argmax(-1)
                non_pad = trg != pad_idx
                correct = (preds[non_pad] == trg[non_pad]).sum().item()
                total = non_pad.sum().item()

                # Update metrics
                epoch_train_loss += loss.item()
                epoch_train_correct += correct
                epoch_train_total += total

                # Collect for BLEU (ignore padding and special tokens)
                for i in range(trg.size(0)):
                    ref = [tgt_idx2word[idx.item()] for idx in trg[i]
                          if idx.item() not in [pad_idx, sos_idx, eos_idx]]
                    hyp = [tgt_idx2word[idx.item()] for idx in preds[i]
                          if idx.item() not in [pad_idx, sos_idx, eos_idx]]
                    references.append([ref])
                    hypotheses.append(hyp)

                # Update progress bar
                t.set_postfix({
                    'loss': epoch_train_loss / (t.n + 1),
                    'acc': epoch_train_correct / max(1, epoch_train_total),
                    'bleu': corpus_bleu(references, hypotheses) * 100
                })

        # Calculate training metrics
        train_loss = epoch_train_loss / len(train_loader)
        train_acc = epoch_train_correct / max(1, epoch_train_total)
        train_bleu = corpus_bleu(references, hypotheses) * 100

        # Validation phase
        encoder.eval()
        decoder.eval()
        epoch_val_loss = 0
        epoch_val_correct = 0
        epoch_val_total = 0
        val_references = []
        val_hypotheses = []

        with torch.no_grad(), tqdm(val_loader, unit="batch", desc=f"Epoch {epoch}/{n_epochs} [Val]") as t:
            for src, trg in t:
                src, trg = src.to(device), trg.to(device)

                # Forward pass
                encoder_concepts, (hidden, cell) = encoder(src)
                outputs, _ = decoder(trg, encoder_concepts, hidden, cell)

                # Calculate loss
                loss = criterion(
                    outputs.view(-1, outputs.shape[-1]),
                    trg.view(-1)
                )

                # Calculate accuracy
                preds = outputs.argmax(-1)
                non_pad = trg != pad_idx
                correct = (preds[non_pad] == trg[non_pad]).sum().item()
                total = non_pad.sum().item()

                # Update metrics
                epoch_val_loss += loss.item()
                epoch_val_correct += correct
                epoch_val_total += total

                # Collect for BLEU
                for i in range(trg.size(0)):
                    ref = [tgt_idx2word[idx.item()] for idx in trg[i]
                          if idx.item() not in [pad_idx, sos_idx, eos_idx]]
                    hyp = [tgt_idx2word[idx.item()] for idx in preds[i]
                          if idx.item() not in [pad_idx, sos_idx, eos_idx]]
                    val_references.append([ref])
                    val_hypotheses.append(hyp)

                # Update progress bar
                t.set_postfix({
                    'loss': epoch_val_loss / (t.n + 1),
                    'acc': epoch_val_correct / max(1, epoch_val_total),
                    'bleu': corpus_bleu(val_references, val_hypotheses) * 100
                })

        # Calculate validation metrics
        val_loss = epoch_val_loss / len(val_loader)
        val_acc = epoch_val_correct / max(1, epoch_val_total)
        val_bleu = corpus_bleu(val_references, val_hypotheses) * 100

        # Update history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['train_bleu'].append(train_bleu)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['val_bleu'].append(val_bleu)

        # Print epoch summary
        print(f"\nEpoch {epoch} Summary:")
        print(f"Train Loss: {train_loss:.4f} | Acc: {train_acc:.2%} | BLEU: {train_bleu:.2f}")
        print(f"Val Loss: {val_loss:.4f} | Acc: {val_acc:.2%} | BLEU: {val_bleu:.2f}")

        # Checkpointing and early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement = 0
            if checkpoint_path:
                torch.save({
                    'epoch': epoch,
                    'encoder_state': encoder.state_dict(),
                    'decoder_state': decoder.state_dict(),
                    'optimizer_state': optimizer.state_dict(),
                    'loss': val_loss,
                    'history': history
                }, checkpoint_path)
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print(f"\nEarly stopping after {patience} epochs without improvement")
                break

    return history

# Step 13: Training Loop

In [78]:
input_dim = len(vocab_fr)
output_dim = len(vocab_en)
emb_dim = 300
enc_hid_dim = 512
dec_hid_dim = 512

embedding_matrix_fr_tensor = torch.FloatTensor(embedding_matrix_fr).to(config.DEVICE)
embedding_matrix_en_tensor = torch.FloatTensor(embedding_matrix_en).to(config.DEVICE)

encoder = Encoder(
    input_dim=input_dim,
    emb_dim=emb_dim,
    enc_hid_dim=enc_hid_dim,
    pretrained_embeddings=embedding_matrix_fr_tensor,
    dropout=0.3
).to(config.DEVICE)

decoder = Decoder(
    output_dim=output_dim,
    emb_dim=emb_dim,
    dec_hid_dim=dec_hid_dim,
    pretrained_embeddings=embedding_matrix_en_tensor,
    dropout=0.3,
    teacher_forcing_ratio=0.5
).to(config.DEVICE)

optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()))
criterion = nn.CrossEntropyLoss(ignore_index=vocab_en['<PAD>'])

# Start training
history = train_model(
    encoder=encoder,
    decoder=decoder,
    train_loader=train_loader,
    val_loader=val_loader,
    src_vocab=vocab_fr,
    tgt_vocab=vocab_en,
    optimizer=optimizer,
    criterion=criterion,
    device=config.DEVICE,
    n_epochs=20,
    max_grad_norm=1.0,
    checkpoint_path='best_model.pt',
    patience=3
)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(
Epoch 1/20 [Train]:   0%|          | 0/29 [00:00<?, ?batch/s]/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Epoch 1/20 [Train]:   0%|          | 0/29 [00:07<?, ?batch/s]


RuntimeError: Expected hidden[0] size (1, 64, 512), got [1, 231, 300]